In [52]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [101]:
#Get All Years into Single DataFrame
import glob
import os
path = os.getcwd() + '/AllYears/'
allFiles = glob.glob(path + "/*.csv")

In [136]:
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    
    #some years have tabs at end of column names
    df.columns = [i.strip('\n') for i in df.columns]
    
    
    #Make sure SALE DATE column is a Date Type
    df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])
    
    #create year column
    df['YEAR'] = df['SALE DATE'].apply(lambda r: r.year)
    list_.append(df)
rawdf = pd.concat(list_,)

In [137]:
rawdf.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
0,0,2,BATHGATE,01 ONE FAMILY HOMES,1,3030,70,,A1,4445 PARK AVENUE,,10457,1,0,1,1694,1497,1899,1,A1,0,2010-01-15,2010
1,1,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,,S1,441 EAST 178 STREET,,10457,1,1,2,1287,2378,1899,1,S1,116000,2010-07-01,2010
2,2,2,BATHGATE,01 ONE FAMILY HOMES,1,3037,42,,A1,4428 PARK AVENUE,,10457,1,0,1,3525,1340,1899,1,A1,287000,2010-12-10,2010
3,3,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,,A1,467 EAST 185 STREET,,10458,1,0,1,1667,1296,1910,1,A1,150000,2010-04-20,2010
4,4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,,A1,2085 BATHGATE AVENUE,,10457,1,0,1,2060,1629,1899,1,A1,145915,2010-07-27,2010


In [147]:
#cleanup DataFrame
#columns to keep
keep =(
[u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING CLASS CATEGORY',
 u'TAX CLASS AT PRESENT', u'BLOCK', u'LOT',
       u'BUILDING CLASS AT PRESENT', u'ADDRESS',
       u'ZIP CODE', u'RESIDENTIAL UNITS', u'COMMERCIAL UNITS', u'TOTAL UNITS',
       u'LAND SQUARE FEET', u'GROSS SQUARE FEET', u'YEAR BUILT',
       u'TAX CLASS AT TIME OF SALE', u'BUILDING CLASS AT TIME OF SALE',
       u'SALE PRICE', u'SALE DATE', 'YEAR'])

df = rawdf[keep].reset_index(drop=True)

df = df[df['SALE PRICE'] > 1000]

In [149]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,YEAR
1,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,S1,441 EAST 178 STREET,10457,1,1,2,1287,2378,1899,1,S1,116000,2010-07-01,2010
2,2,BATHGATE,01 ONE FAMILY HOMES,1,3037,42,A1,4428 PARK AVENUE,10457,1,0,1,3525,1340,1899,1,A1,287000,2010-12-10,2010
3,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,A1,467 EAST 185 STREET,10458,1,0,1,1667,1296,1910,1,A1,150000,2010-04-20,2010
4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,A1,2085 BATHGATE AVENUE,10457,1,0,1,2060,1629,1899,1,A1,145915,2010-07-27,2010
5,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,42,A1,2069 BATHGATE AVENUE,10457,1,0,1,1964,1424,1899,1,A1,72790,2010-06-03,2010


In [151]:
df.count()

BOROUGH                           267379
NEIGHBORHOOD                      267379
BUILDING CLASS CATEGORY           267379
TAX CLASS AT PRESENT              267379
BLOCK                             267379
LOT                               267379
BUILDING CLASS AT PRESENT         267379
ADDRESS                           267379
ZIP CODE                          267379
RESIDENTIAL UNITS                 267379
COMMERCIAL UNITS                  267379
TOTAL UNITS                       267379
LAND SQUARE FEET                  267379
GROSS SQUARE FEET                 267379
YEAR BUILT                        267379
TAX CLASS AT TIME OF SALE         267379
BUILDING CLASS AT TIME OF SALE    267379
SALE PRICE                        267379
SALE DATE                         267379
YEAR                              267379
dtype: int64

In [193]:
#Zip Codes to Keep
groups = (df.groupby('ZIP CODE').count() > 500)
zips = groups[groups['BLOCK'] == True].index

In [206]:
clean = df[df['ZIP CODE'].isin(zips)]

In [217]:
clean.groupby(['ZIP CODE']).size()

ZIP CODE
10001     1094
10002     1469
10003     3352
10005      725
10007      713
10009      670
10010     2115
10011     4390
10012     1234
10013     2455
10014     2133
10016     3795
10017     1819
10019    15769
10021     3420
10022     3683
10023     5279
10024     3302
10025     3421
10026     1230
10027     1253
10028     2733
10029      810
10031      753
10032      588
10033      683
10035      677
10036     1397
10038      935
10040      555
         ...  
11375     4067
11377     2088
11378     1274
11379     1223
11385     2207
11411      605
11412     1639
11413     1441
11414     1409
11415      767
11416      693
11417      913
11418     1052
11419      989
11420     1476
11421     1270
11422      926
11423      894
11426      789
11427      827
11428      518
11429      895
11432     1281
11433     1600
11434     2131
11435     1443
11436     1080
11691     1064
11692      577
11694      757
dtype: int64